In [33]:
import os
import sys
from functools import partial
import warnings

import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from sklearn.externals import joblib
%matplotlib inline
import seaborn as sns

sys.path.append('../')
from src.utils import parallel_apply
from src.feature_extraction import add_features_in_group

warnings.filterwarnings('ignore')

DIR = '/mnt/ml-team/minerva/open-solutions/home-credit'
description = pd.read_csv(os.path.join(DIR,'data/HomeCredit_columns_description.csv'),encoding = 'latin1')
application = pd.read_csv(os.path.join(DIR, 'files/unzipped_data/application_train.csv'))
bureau = pd.read_csv(os.path.join(DIR, 'files/unzipped_data/bureau.csv'))

In [34]:
bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


# Preprocessing
## Solution 3

In [3]:
(bureau['AMT_CREDIT_SUM'] == 0).sum()

66582

This shows that imputing with nan with 0 is probably a bad idea

# Feature Engineering
## Solution 3
### Hand crafted features

In [ ]:
bureau[bureau['SK_ID_CURR']==215354]

### First build helper columns

In [35]:
bureau['bureau_credit_active_binary'] = (bureau['CREDIT_ACTIVE'] != 'Closed').astype(int)
bureau['bureau_credit_enddate_binary'] = (bureau['DAYS_CREDIT_ENDDATE'] > 0).astype(int)

groupby_SK_ID_CURR = bureau.groupby(by=['SK_ID_CURR'])

In [ ]:
description[description['Row'] == 'DAYS_CREDIT'].Description.tolist()

In [ ]:
features = pd.DataFrame({'SK_ID_CURR':bureau['SK_ID_CURR'].unique()})
features.head()

In [ ]:
group_object = groupby_SK_ID_CURR['DAYS_CREDIT'].agg('count').reset_index()
group_object.rename(index=str, columns={'DAYS_CREDIT': 'bureau_number_of_past_loans'},inplace=True)

features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
features.head()

In [ ]:
group_object = groupby_SK_ID_CURR['CREDIT_TYPE'].agg('nunique').reset_index()
group_object.rename(index=str, columns={'CREDIT_TYPE': 'bureau_number_of_loan_types'},inplace=True)

features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
features.head()

In [ ]:
features['bureau_average_of_past_loans_per_type'] = \
    features['bureau_number_of_past_loans'] / features['bureau_number_of_loan_types']
    
features.head()

In [ ]:
group_object = groupby_SK_ID_CURR['bureau_credit_active_binary'].agg('mean').reset_index()

features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
features.head()

In [ ]:
group_object = groupby_SK_ID_CURR['AMT_CREDIT_SUM_DEBT'].agg('sum').reset_index()
group_object.rename(index=str, columns={'AMT_CREDIT_SUM_DEBT': 'bureau_total_customer_debt'},inplace=True)

features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
features.head()

In [ ]:
group_object = groupby_SK_ID_CURR['AMT_CREDIT_SUM'].agg('sum').reset_index()
group_object.rename(index=str, columns={'AMT_CREDIT_SUM': 'bureau_total_customer_credit'},inplace=True)

features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
features.head()

In [ ]:
features['bureau_debt_credit_ratio'] = \
    features['bureau_total_customer_debt'] / features['bureau_total_customer_credit']
    
features.head()

In [ ]:
group_object = groupby_SK_ID_CURR['AMT_CREDIT_SUM_OVERDUE'].agg('sum').reset_index()
group_object.rename(index=str, columns={'AMT_CREDIT_SUM_OVERDUE': 'bureau_total_customer_overdue'},inplace=True)

features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
features.head()

In [ ]:
features['bureau_overdue_debt_ratio'] = \
    features['bureau_total_customer_overdue'] / features['bureau_total_customer_debt']
    
features.head()

In [ ]:
group_object = groupby_SK_ID_CURR['CNT_CREDIT_PROLONG'].agg('sum').reset_index()
group_object.rename(index=str, columns={'CNT_CREDIT_PROLONG': 'bureau_average_creditdays_prolonged'},inplace=True)

features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
features.head()

In [ ]:
group_object = groupby_SK_ID_CURR['bureau_credit_enddate_binary'].agg('mean').reset_index()
group_object.rename(index=str, columns={'bureau_credit_enddate_binary': 'bureau_credit_enddate_percentage'},inplace=True)

features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
features.head()

In [ ]:
bureau_ONE = features[features['SK_ID_CURR']==215354]

In [ ]:
bureau_ONE

In [ ]:
application = application.merge(features,
                                left_on=['SK_ID_CURR'],
                                right_on=['SK_ID_CURR'],
                                how='left',
                                validate='one_to_one')

In [ ]:
engineered_numerical_columns = list(features.columns)
engineered_numerical_columns.remove('SK_ID_CURR')
bureau_eng = application[engineered_numerical_columns + ['TARGET']]
bureau_eng_corr = abs(bureau_eng.corr())

In [ ]:
bureau_eng_corr.sort_values('TARGET', ascending=False)['TARGET']

In [ ]:
sns.heatmap(bureau_eng_corr, 
            xticklabels=bureau_eng_corr.columns,
            yticklabels=bureau_eng_corr.columns)

## Aggregations

In [ ]:
BUREAU_AGGREGATION_RECIPIES = [('CREDIT_TYPE', 'count'),
                               ('CREDIT_ACTIVE', 'size')
                               ]
for agg in ['mean', 'min', 'max', 'sum', 'var']:
    for select in ['AMT_ANNUITY',
                   'AMT_CREDIT_SUM',
                   'AMT_CREDIT_SUM_DEBT',
                   'AMT_CREDIT_SUM_LIMIT',
                   'AMT_CREDIT_SUM_OVERDUE',
                   'AMT_CREDIT_MAX_OVERDUE',
                   'CNT_CREDIT_PROLONG',
                   'CREDIT_DAY_OVERDUE',
                   'DAYS_CREDIT',
                   'DAYS_CREDIT_ENDDATE',
                   'DAYS_CREDIT_UPDATE'
                   ]:
        BUREAU_AGGREGATION_RECIPIES.append((select, agg))
BUREAU_AGGREGATION_RECIPIES = [(['SK_ID_CURR'], BUREAU_AGGREGATION_RECIPIES)]

In [ ]:
groupby_aggregate_names = []
for groupby_cols, specs in tqdm(BUREAU_AGGREGATION_RECIPIES):
    group_object = bureau.groupby(groupby_cols)
    for select, agg in tqdm(specs):
        groupby_aggregate_name = '{}_{}_{}'.format('_'.join(groupby_cols), agg, select)
        application = application.merge(group_object[select]
                              .agg(agg)
                              .reset_index()
                              .rename(index=str,
                                      columns={select: groupby_aggregate_name})
                              [groupby_cols + [groupby_aggregate_name]],
                              on=groupby_cols,
                              how='left')
        groupby_aggregate_names.append(groupby_aggregate_name)

In [ ]:
application.head()

In [ ]:
application_agg = application[groupby_aggregate_names + ['TARGET']]
application_agg_corr = abs(application_agg.corr())

In [ ]:
application_agg_corr.sort_values('TARGET', ascending=False)['TARGET']

# Solution 4
## Hand Crafted Features

In [4]:
COLUMN_NAME = 'DAYS_CREDIT_UPDATE'
d = description[description['Row']==COLUMN_NAME]
display(d.Description.tolist())
display(d.Special.tolist())

['How many days before loan application did last information about the Credit Bureau credit come']

['time only relative to the application']

In [6]:
positive_ID = application[application['TARGET']==1]['SK_ID_CURR'].tolist()
positive_ID[:4]

[100002, 100031, 100047, 100049]

In [7]:
value_counts = bureau[bureau['SK_ID_CURR'].isin(positive_ID)]['SK_ID_CURR'].value_counts()

In [8]:
value_counts.head()

280155    55
329272    48
371490    47
349795    43
376299    41
Name: SK_ID_CURR, dtype: int64

In [21]:
bureau_one = bureau[bureau['SK_ID_CURR']==280155].sort_values(['DAYS_CREDIT'],ascending=False)

In [22]:
bureau_one.head(10)

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,bureau_credit_active_binary,bureau_credit_enddate_binary,days_credit_diff
428795,280155,6832771,Active,currency 1,-3,0,59.0,NaN,NaN,0,9000.0,9090.0,NaN,0.0,Consumer credit,0,0.0,1,1,2599.0
428793,280155,6832769,Closed,currency 1,-26,0,35.0,-4.0,NaN,0,13500.0,0.0,NaN,0.0,Consumer credit,-4,0.0,0,1,-23.0
428794,280155,6832770,Active,currency 1,-39,0,140.0,NaN,NaN,0,121203.0,NaN,NaN,0.0,Consumer credit,-3,0.0,1,1,-13.0
428792,280155,6832768,Closed,currency 1,-52,0,9.0,-25.0,NaN,0,13500.0,0.0,NaN,0.0,Consumer credit,-25,0.0,0,1,-13.0
522591,280155,6143520,Closed,currency 1,-78,0,-17.0,-53.0,NaN,0,13500.0,0.0,NaN,0.0,Consumer credit,-53,0.0,0,0,-26.0
184677,280155,6832767,Closed,currency 1,-97,0,-36.0,-84.0,NaN,0,4500.0,0.0,NaN,0.0,Consumer credit,-84,NaN,0,0,-19.0
428791,280155,6832766,Closed,currency 1,-155,0,-93.0,-130.0,NaN,0,4500.0,0.0,NaN,0.0,Consumer credit,-130,0.0,0,0,-58.0
428790,280155,6832765,Closed,currency 1,-204,0,-143.0,-179.0,NaN,0,22500.0,0.0,NaN,0.0,Consumer credit,-179,NaN,0,0,-49.0
428805,280155,6832782,Closed,currency 1,-219,0,-189.0,-183.0,0.0,0,22500.0,0.0,0.0,0.0,Consumer credit,-135,0.0,0,0,-15.0
184676,280155,6832764,Closed,currency 1,-236,0,-175.0,-205.0,NaN,0,18000.0,0.0,NaN,0.0,Consumer credit,-205,NaN,0,0,-17.0


In [36]:
bureau_one['days_credit_diff'] = bureau_one['DAYS_CREDIT'].diff().replace(np.nan, 0,)
bureau_one

NameError: name 'bureau_one' is not defined

In [38]:
bureau_ = bureau.copy()#sample(10000)

In [39]:
features = pd.DataFrame({'SK_ID_CURR':bureau_['SK_ID_CURR'].unique()})

In [40]:
bureau_.sort_values(['SK_ID_CURR','DAYS_CREDIT'],ascending=False, inplace=True)
bureau_['days_credit_diff'] = bureau_['DAYS_CREDIT'].diff().replace(np.nan, 0)    

groupby_SK_ID_CURR = bureau_.groupby(by=['SK_ID_CURR'])

In [41]:
group_object = groupby_SK_ID_CURR['bureau_credit_active_binary'].agg('sum').reset_index()
group_object.rename(index=str, columns={'bureau_credit_active_count': 'bureau_credit_active_binary'},inplace=True)

features = features.merge(group_object, on=['SK_ID_CURR'], how='left')

In [42]:
group_object = groupby_SK_ID_CURR['SK_ID_BUREAU'].agg('nunique').reset_index()
group_object.rename(index=str, columns={'SK_ID_BUREAU': 'bureau_query_count'},inplace=True)

features = features.merge(group_object, on=['SK_ID_CURR'], how='left')

In [43]:
features.head()

,SK_ID_CURR,bureau_credit_active_binary,bureau_query_count
0,215354,6,11
1,162297,3,6
2,402440,1,1
3,238881,3,8
4,222183,5,8


In [44]:
def trend_in_last_k_bureau_features(gr, periods):
    gr_ = gr.copy()
    gr_['days_credit_diff'].iloc[0] = 0

    features = {}

    for period in periods:
        gr_period = gr_.iloc[:period]
        features = _add_trend_feature(features,gr_period,
                                      'days_credit_diff','{}_period_trend_'.format(period)
                                     )
    return pd.Series(features)

def last_k_bureau_features(gr, periods):
    gr_ = gr.copy()
    gr_['days_credit_diff'].iloc[0] = 0 
    features = {}
    for period in periods:
        gr_period = gr_.iloc[:period]

        features = add_features_in_group(features,gr_period, 'days_credit_diff', 
                                       ['mean','max','min','std', 'median','skew', 'kurt','iqr'],
                                         'last_{}_'.format(period))
            
    return pd.Series(features)

def _add_trend_feature(features,gr,feature_name, prefix):
    y = gr[feature_name].values
    try:
        x = np.arange(0,len(y)).reshape(-1,1)
        lr = LinearRegression()
        print('fitting')
        lr.fit(x,y)
        trend = lr.coef_[0]
    except:
        trend=np.nan
    features['{}{}'.format(prefix,feature_name)] = trend
    return features

In [25]:
func = partial(trend_in_last_k_bureau_features, periods=[50])

g = parallel_apply(groupby_SK_ID_CURR, func, index_name='SK_ID_CURR',
                   num_workers=4, chunk_size=10000).reset_index()
features = features.merge(g, on='SK_ID_CURR', how='left')

display(features.head())




  0%|          | 0/1.0 [00:00<?, ?it/s]Process ForkPoolWorker-19:
Process ForkPoolWorker-18:


KeyboardInterrupt: 

In [45]:
func = partial(last_k_bureau_features, periods=[5, 10, 50])

g = groupby_SK_ID_CURR.apply(func).reset_index()
# g = parallel_apply(groupby_SK_ID_CURR, func, index_name='SK_ID_CURR',
#                    num_workers=4, chunk_size=100000).reset_index()
features = features.merge(g, on='SK_ID_CURR', how='left')

display(features.head())

,SK_ID_CURR,bureau_credit_active_binary,bureau_query_count,last_10_days_credit_diff_mean,last_50_days_credit_diff_mean,last_5_days_credit_diff_mean
0,215354,6,11,-169.100000,-166.272727,-46.0
1,162297,3,6,-363.166667,-363.166667,-323.8
2,402440,1,1,0.000000,0.000000,0.0
3,238881,3,8,-352.000000,-352.000000,-69.8
4,222183,5,8,-303.625000,-303.625000,-93.8


In [46]:
features.shape

(305811, 6)

In [47]:
application = application.merge(features,
                                left_on=['SK_ID_CURR'],
                                right_on=['SK_ID_CURR'],
                                how='left',
                                validate='one_to_one')

In [48]:
feature_names = list(features.columns)
feature_names.remove('SK_ID_CURR')
X = application[feature_names + ['TARGET']]
feature_corr_abs = abs(X.corr())

In [49]:
feature_corr_abs.sort_values('TARGET', ascending=False)['TARGET']

TARGET                           1.000000
bureau_credit_active_binary      0.067624
last_5_days_credit_diff_mean     0.044616
last_10_days_credit_diff_mean    0.043493
last_50_days_credit_diff_mean    0.042364
bureau_query_count               0.004056
Name: TARGET, dtype: float64

In [ ]:
# group = bureau[bureau['bureau_credit_enddate_binary'] == 1].groupby(
#     by=['SK_ID_CURR']).apply(
#     lambda x: x.sort_values(['DAYS_CREDIT_ENDDATE'], ascending=True)).reset_index(drop=True)
# group['bureau_days_enddate_diff'] = group.groupby(by=['SK_ID_CURR'])['DAYS_CREDIT_ENDDATE'].diff()
# group['bureau_days_enddate_diff'] = group['bureau_days_enddate_diff'].fillna(0).astype('uint32')

# bureau = bureau.merge(group[['bureau_days_enddate_diff', 'SK_ID_BUREAU']], on=['SK_ID_BUREAU'], how='left')
# bureau['bureau_average_enddate_future'] = bureau.groupby(
#     by=['SK_ID_CURR'])['bureau_days_enddate_diff'].agg('mean').reset_index()['bureau_days_enddate_diff']

# bureau['bureau_days_credit_diff'] = bureau.groupby(
#     by=['SK_ID_CURR']).apply(
#     lambda x: x.sort_values(['DAYS_CREDIT'], ascending=False)).reset_index(drop=True)['DAYS_CREDIT']
# bureau['bureau_days_credit_diff'] *= -1
# bureau['bureau_days_credit_diff'] = bureau.groupby(by=['SK_ID_CURR'])['bureau_days_credit_diff'].diff()
# bureau['bureau_days_credit_diff'] = bureau['bureau_days_credit_diff'].fillna(0)